<a href="https://colab.research.google.com/github/Nburkhal/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/LS_DS_432_Convolution_Neural_Networks_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

ResNet50 = ResNet50(weights='imagenet')
features = model.predict(x)

```

Next you will need to remove the last layer from the ResNet model. Here, we loop over the layers to use the sequential API. There are easier ways to add and remove layers using the Keras functional API, but doing so introduces other complexities. 

```python
# Remote the Last Layer of ResNEt
ResNet50._layers.pop(0)

# Out New Model
model = Sequential()

# Add Pre-trained layers of Old Model to New Model
for layer in ResNet50.layers:
    model.add(layer)

# Turn off additional training of ResNet Layers for speed of assignment
for layer in model.layers:
    layer.trainable = False

# Add New Output Layer to Model
model.add(Dense(1, activation='sigmoid'))
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

In [26]:
### YOUR CODE HERE
# mount drive to colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
# there's a db file in them mountains! (manually removed)
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.python.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import random
import os
import cv2
import matplotlib.pyplot as plt


# Load images into numpy arrays/ create y for labels
FILEPATH = '/content/drive/My Drive/Colab Notebooks/module2-convolutional-neural-networks/data'
CATEGORIES = ['forest', 'mountain']
training_data = []
IMG_SIZE = 224

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(FILEPATH, category) # path to forest/mountains dir
        class_num = CATEGORIES.index(category)
        print(path, class_num)
        for img in os.listdir(path):
            print(img)
            try:
                if not img.startswith(".jpg"):
                    img_array = cv2.imread(os.path.join(path, img))
                    new_img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    training_data.append([new_img_array, class_num])
            except Exception as e:
                pass
    X = []
    y = []

    for features, label in training_data:
        X.append(features)
        y.append(label)

    X = np.array(X)
    y = to_categorical(np.array(y))

    print('------- Shape of X, y data -------')
    print(X.shape, y.shape)

    return X, y

X, y = create_training_data()

/content/drive/My Drive/Colab Notebooks/module2-convolutional-neural-networks/data/forest 0
nat147.jpg
cdmc430.jpg
natu977.jpg
Thumbs.db
bost103.jpg
nat146.jpg
for52.jpg
text20.jpg
bost102.jpg
for47.jpg
for84.jpg
land215.jpg
for136.jpg
cdmc547.jpg
for93.jpg
for87.jpg
for78.jpg
for50.jpg
for3.jpg
for44.jpg
land376.jpg
bost101.jpg
natu949.jpg
land809.jpg
nat347.jpg
cdmc368.jpg
nat1184.jpg
nat145.jpg
nat623.jpg
bost100.jpg
land377.jpg
for79.jpg
nat1033.jpg
for86.jpg
natu169.jpg
nat1027.jpg
land217.jpg
for137.jpg
natu425.jpg
land765.jpg
for121.jpg
text21.jpg
natu396.jpg
text35.jpg
nat219.jpg
natu31.jpg
for130.jpg
land760.jpg
nat354.jpg
cdmc352.jpg
natc12.jpg
nat619.jpg
nat221.jpg
nat209.jpg
natu423.jpg
for119.jpg
cdmc556.jpg
for127.jpg
nat223.jpg
for96.jpg
for82.jpg
cdmc12.jpg
natu794.jpg
land159.jpg
cdmc344.jpg
cdmc351.jpg
tell51.jpg
sclos13.jpg
text26.jpg
for132.jpg
natu424.jpg
natu430.jpg
natc13.jpg
nat442.jpg
nat220.jpg
land102.jpg
nat867.jpg
land314.jpg
nat697.jpg
natu863.jpg
cdmc385.

In [28]:
# Train-test split the data
X_train, X_test, y_train, y_test = train_test_split(X,
	                                                y,
                                                    test_size=0.33)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((471, 224, 224, 3), (232, 224, 224, 3), (471, 2), (232, 2))

In [0]:
# Load pre-trained resnet from keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense

res = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

for layer in res.layers:
    layer.trainable = False
    
# add your head on top
x = res.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=res.input, outputs=predictions)

In [0]:
# Compile model
model.compile(loss="categorical_crossentropy", 
              optimizer='adam', 
              metrics=["accuracy"])

In [37]:
EPOCHS = 5
BATCH_SIZE = 10

history = model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.15, batch_size=BATCH_SIZE, verbose=1)

Train on 400 samples, validate on 71 samples
Epoch 1/5
400/400 [==============================] - 105s 263ms/sample - loss: 0.6988 - acc: 0.8250 - val_loss: 0.0143 - val_acc: 1.0000
Epoch 2/5
400/400 [==============================] - 103s 256ms/sample - loss: 0.0923 - acc: 0.9700 - val_loss: 0.0605 - val_acc: 0.9718
Epoch 3/5
400/400 [==============================] - 103s 257ms/sample - loss: 0.1854 - acc: 0.9675 - val_loss: 0.2487 - val_acc: 0.9155
Epoch 4/5
400/400 [==============================] - 102s 255ms/sample - loss: 0.5530 - acc: 0.8550 - val_loss: 0.3539 - val_acc: 0.8592
Epoch 5/5
400/400 [==============================] - 103s 257ms/sample - loss: 0.1542 - acc: 0.9375 - val_loss: 0.0079 - val_acc: 1.0000


In [39]:
score = model.evaluate(X_test, y_test, verbose=False)
print('---------------Validation Metrics---------------')
print('Loss:', score[0])
print('Accuracy:', score[1])

---------------Validation Metrics---------------
Loss: 0.033870818093419075
Accuracy: 0.98706895


In [44]:
y_pred = model.predict(X_test)
y_pred = (y_pred > .5).astype(int)
y_pred[:25]

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1]])

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9870689655172413

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language